# Setup

## Imports 

In [0]:
from PIL import Image
from sklearn import utils
from sklearn.model_selection import ParameterGrid
from sklearn.model_selection import train_test_split
from torch.autograd import Function
from torch.backends import cudnn
from torch.utils.data import DataLoader
from torch.utils.data import Subset, DataLoader
from torch.utils.model_zoo import load_url as load_state_dict_from_url
from torchvision import models
from torchvision import transforms
from torchvision.datasets import ImageFolder
from torchvision.datasets import VisionDataset
from torchvision.models import alexnet
from torchvision.transforms.functional import pad
from tqdm import tqdm
import logging
import matplotlib.pyplot as plt
import numbers
import numpy as np
import os
import os.path
import shutil
import sys
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
import zipfile


## Drive mount


In [0]:
from google.colab import drive
drive.mount('/content/drive')

## File extraction and setup

In [0]:
if not os.path.exists("/content/ROD.zip"):
  !cp -r "/content/drive/My Drive/DL_project/ROD.zip" "/content/"
if not os.path.exists("/content/synROD.zip"):
  !cp -r "/content/drive/My Drive/DL_project/synROD.zip" "/content/"

## Path variables declaration and download

In [0]:
rod_path = "/content/ROD/ROD"
synrod_path = "/content/synROD/synROD"
rod_destination_path = "/content/ROD"
synrod_destination_path = "/content/synROD"

if not os.path.isdir(rod_destination_path) and not os.path.exists("/content/ROD.zip"):
  # ROD 
  # https://drive.google.com/open?id=1p1GORdB44NjtNWJ4d1xqttseM1X9lWNF
  # https://drive.google.com/open?id=168neCvaHwMffFOqjOkth-wVaP4tRFuSW
  !curl -c ./cookie -s -L "https://drive.google.com/uc?export=download&id=168neCvaHwMffFOqjOkth-wVaP4tRFuSW" > /dev/null
  !curl -Lb ./cookie "https://drive.google.com/uc?export=download&confirm=`awk '/download/ {print $NF}' ./cookie`&id=168neCvaHwMffFOqjOkth-wVaP4tRFuSW" -o "ROD.zip"

if not os.path.isdir(synrod_destination_path) and  not os.path.exists("/content/synROD.zip"):
  # synROD 
  # https://drive.google.com/open?id=1rry4GViJLmmMpbm0B2s7MyQs5Dx8pFS3
  # https://drive.google.com/open?id=1V1fthSNAvsPRF6hLt_kf_xonw7lxAV03
  !curl -c ./cookie -s -L "https://drive.google.com/uc?export=download&id=1V1fthSNAvsPRF6hLt_kf_xonw7lxAV03" > /dev/null
  !curl -Lb ./cookie "https://drive.google.com/uc?export=download&confirm=`awk '/download/ {print $NF}' ./cookie`&id=1V1fthSNAvsPRF6hLt_kf_xonw7lxAV03" -o "synROD.zip"
# Extract ROD dataset
if not os.path.isdir(rod_destination_path):
  with zipfile.ZipFile("/content/ROD.zip", 'r') as zip_ref:
      zip_ref.extractall(rod_destination_path)

# Extract synROD dataset
if not os.path.isdir(synrod_destination_path):
  with zipfile.ZipFile("/content/synROD.zip", 'r') as zip_ref:
      zip_ref.extractall(synrod_destination_path)

In [0]:
"""
# Extract ROD dataset
rod_path = "/content/ROD/ROD"
synrod_path = "/content/synROD/synROD"

rod_destination_path = "/content/ROD"
if not os.path.isdir(rod_destination_path):
  with zipfile.ZipFile("/content/ROD.zip", 'r') as zip_ref:
      zip_ref.extractall(rod_destination_path)

# Extract synROD dataset
synrod_destination_path = "/content/synROD"
if not os.path.isdir(synrod_destination_path):
  with zipfile.ZipFile("/content/synROD.zip", 'r') as zip_ref:
      zip_ref.extractall(synrod_destination_path)
"""

## Copy in current folder datasets and net classes

In [0]:
!cp -r "/content/drive/My Drive/inception_models/dataset/." "/content/"
!cp -r "/content/drive/My Drive/inception_models/net/." "/content/"
!cp -r "/content/drive/My Drive/inception_models/transform_config/." "/content/"
!cp -r "/content/drive/My Drive/inception_models/splits/." "/content/"

## Import datasets, net and configurator classes

In [0]:
from synrod import SynRODMOD
from rod import RODMOD
from rod_utils import *
from dcepnet_concat_kaiming import DCepNet
from tconfig import TransformConfig

# Train test rotation function definition

In [0]:
def train_test_ours(synrod, synrod_validation, rod, hyperparams, light_validation=False):
  """Train the architecture called "OURS" in the reference paper with rotation pretext task.
  The net is trained in end-to-end fashion.

  Args:
    synrod: train dataset
    rod: test dataset
    hyperparams: parameters dict with the keys
      {
        lr
        batch_size
        weight_decay 
        step_size
        epochs 
        lambda (!!!)
        momentum (optional, default 0.9)
        gamma (optional, default 0.1)
      }
    light_validation: if True the validation is done only in the last epoch.

  Return: 
    (trained_model, train_loss, train_acc, test_loss, test_acc).
  """
  lr = hyperparams["lr"]
  batch_size = hyperparams["batch_size"]
  weight_decay = hyperparams["weight_decay"]
  step_size = hyperparams["step_size"]
  epochs = hyperparams["epochs"]
  curr_momentum = hyperparams.get("momentum", 0.9)
  curr_gamma = hyperparams.get("gamma", 0.1) 
  lambda_ = hyperparams["lambda"]

  decay_policy = hyperparams["lr_epoch_num_decay_policy"]
  use_fixed_stepsize = decay_policy == None

  em_weight = 0.1
  
  DEVICE = "cuda"
  cudnn.benchmark

  # dataloader definition with given batch size
  source = DataLoader(synrod,  batch_size=batch_size, shuffle=True, drop_last=True, num_workers=4, collate_fn=collate)
  source_validation = DataLoader(synrod_validation,  batch_size=batch_size, shuffle=True, drop_last=True, num_workers=4, collate_fn=collate)
  source_rot = DataLoader(synrod_train,  batch_size=batch_size, shuffle=True, drop_last=True, num_workers=4, collate_fn=collate)
  target_rot = DataLoader(rod,  batch_size=batch_size, shuffle=True, drop_last=True, num_workers=4, collate_fn=collate)
  target = DataLoader(rod,  batch_size=batch_size, shuffle=True, drop_last=True, num_workers=4, collate_fn=collate)

  # NET DEFINITION
  net = DCepNet(num_classes=47, pretext_classes=4).to("cuda")

  criterion = nn.CrossEntropyLoss() 
  entropy_min_criterion = HLoss()

  parameters_to_optimize = net.parameters() 
  optimizer = optim.SGD(parameters_to_optimize, lr=lr, 
                            momentum=curr_momentum, 
                            weight_decay=weight_decay)
  scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=step_size, gamma=curr_gamma)

  # lists that accumulate loss/accuracy values over the training period
  train_loss = []
  train_acc = []
  validation_loss = []
  validation_acc = []
  test_loss = []
  test_acc = []
  # lambda parameter used to weight the losses of main and pretext task
  next_update_epoch = 1
  slow_schedule_count = 1
  iter_count = 1

  running_corrects = 0
  tot_samples = 0
  for i in range(epochs):
    counter_mod = 0
    n_iters = 0
    train_main_acc_val = 0
    train_main_loss_val = 0

    train_rot_source_acc_val = 0
    train_rot_source_loss_val = 0

    train_rot_target_acc_val = 0
    train_rot_target_loss_val = 0
    
    test_main_acc_val = 0
    test_main_loss_val = 0
    batch_count = 0
    for source_batch, target_batch, source_rot_batch, target_rot_batch in zip(source, target, source_rot, target_rot):
      
      net.train()
      S, _ = format_batch(source_batch, pretext_task="rotation")
      T, _ = format_batch(target_batch, pretext_task="rotation")

      _, S_hat = format_batch(source_rot_batch, pretext_task="rotation")
      _, T_hat = format_batch(target_rot_batch, pretext_task="rotation")

      # zero the gradients
      optimizer.zero_grad() 

      # MAIN TASK
      # setup SOURCE DOMAIN STANDARD dataset to feed to the net
      source_rgb_images = S["rgb"].to(DEVICE)
      source_depth_images = S["depth"].to(DEVICE)
      source_main_labels = S["label"].to(DEVICE)
      # train on source original images 
      outputs, out_aux  = net.forward(source_rgb_images, source_depth_images, mode="main")
      loss_M = criterion(outputs, source_main_labels)
      loss_aux = criterion(out_aux, source_main_labels)
      # compute stats
      _, preds = torch.max(outputs.data, 1)
      running_corrects = torch.sum(preds == source_main_labels.data).data.item()
      tot_samples = len(source_main_labels)
      train_main_loss_val += loss_M.item()
      train_main_acc_val += running_corrects/tot_samples
      


      # entropy minimization
      # setup TARGET DOMAIN STANDARD dataset to feed to the net
      target_rgb_images = T["rgb"].to(DEVICE)
      target_depth_images = T["depth"].to(DEVICE)
      # target labels in this phase can't be used for training
      # train on source original images 
      outputs, out_aux = net.forward(target_rgb_images, target_depth_images, mode="main")
      loss_entropy_min = entropy_min_criterion(outputs)
      loss_ent_aux = entropy_min_criterion(out_aux)


      # PRETEXT TASK 
      # setup  SOURCE DOMAIN ROTATED dataset to feed to the net
      source_rotated_rgb_images = S_hat["rgb"].to(DEVICE)
      source_rotated_depth_images = S_hat["depth"].to(DEVICE)
      source_rotated_labels = S_hat["label"].to(DEVICE)
      # train on source rotated 
      outputs, out_aux = net.forward(source_rotated_rgb_images, source_rotated_depth_images, mode="pretext")
      loss_P_1 = criterion(outputs, source_rotated_labels) 
      loss_src_rot_aux = criterion(out_aux, source_rotated_labels)
      # compute stats
      _, preds = torch.max(outputs.data, 1)
      running_corrects = torch.sum(preds == source_rotated_labels.data).data.item()
      tot_samples = len(source_rotated_labels)
      train_rot_source_acc_val += running_corrects/tot_samples
      train_rot_source_loss_val += loss_P_1.item()



      #setup TARGET DOMAIN ROTATED dataset to feed to the net
      target_rotated_rgb_images = T_hat["rgb"].to(DEVICE)
      target_rotated_depth_images = T_hat["depth"].to(DEVICE)
      target_rotated_labels = T_hat["label"].to(DEVICE)
      # train on target rotated
      outputs, out_aux = net.forward(target_rotated_rgb_images, target_rotated_depth_images, mode="pretext")
      loss_P_2 = criterion(outputs, target_rotated_labels)
      loss_trg_rot_aux = criterion(out_aux, target_rotated_labels)
      # compute stats
      _, preds = torch.max(outputs.data, 1)
      running_corrects = torch.sum(preds == target_rotated_labels.data).data.item()
      tot_samples = len(target_rotated_labels)
      train_rot_target_acc_val += running_corrects/tot_samples
      train_rot_target_loss_val += loss_P_2.item()


      # BACKPROP WITH THE FULL LOSS
      loss = loss_M + 0.3*(loss_aux) +\
         (em_weight/tot_samples)*(loss_entropy_min + 0.3*loss_ent_aux) +\
          lambda_*(loss_P_1 + loss_P_2 + 0.3*(loss_src_rot_aux + loss_trg_rot_aux) )

 
      loss.backward() 
      batch_count += 1
      # UPDATE GRADIENTS accumulated --> batch size 8 --> update each 16 elements 
      if batch_count % 2 == 0:
        optimizer.step()
      # UPDATE GRADIENTS
      #optimizer.step()
      del source_rgb_images , source_depth_images , source_main_labels, target_depth_images, target_rgb_images
      del source_rotated_rgb_images, source_rotated_depth_images, source_rotated_labels
      del target_rotated_rgb_images, target_rotated_depth_images, target_rotated_labels 
      n_iters += 1
    
    train_loss.append(train_main_loss_val/n_iters)
    train_acc.append(train_main_acc_val/n_iters)
    print("EPOCH: ", i + 1)
    print("train main accuracy: ", train_main_acc_val/n_iters)
    print("train main loss: ", train_main_loss_val/n_iters)
    print("train rot source accuracy: ", train_rot_source_acc_val/n_iters)
    print("train rot source loss: ", train_rot_source_loss_val/n_iters)
    print("train rot target accuracy: ", train_rot_target_acc_val/n_iters)
    print("train rot target loss: ", train_rot_target_loss_val/n_iters)
    

    net.eval()
    n_iters = 0
    tot_samples = 0
    validation_corrects = 0
    validation_main_loss_val = 0
    for source_val_batch in source_validation:
      S, _ = format_batch(source_val_batch, pretext_task="rotation")
     
      # MAIN TASK
      # setup SOURCE DOMAIN STANDARD dataset to feed to the net
      source_rgb_images = S["rgb"].to(DEVICE)
      source_depth_images = S["depth"].to(DEVICE)
      source_main_labels = S["label"].to(DEVICE)
      
      outputs = net.forward(source_rgb_images, source_depth_images, mode="main")
      loss_M = criterion(outputs, source_main_labels)
      
      _, preds = torch.max(outputs.data, 1)
      validation_corrects += torch.sum(preds == source_main_labels.data).data.item()
      tot_samples += len(source_main_labels)
      validation_main_loss_val += loss_M.item()
      del source_rgb_images , source_depth_images , source_main_labels
      n_iters += 1
          
    validation_acc.append(validation_corrects/tot_samples)
    validation_loss.append(validation_main_loss_val/n_iters)
    print("validation main accuracy: ", validation_corrects/tot_samples)
    print("validation main loss: ", validation_main_loss_val/n_iters)
    

    # TEST RESULTS OF THE CURRENT BATCH OF TRAINING
    if not light_validation or i == epochs - 1:
      net.eval() 
      n_iters = 0
      for target_batch in target: 
        # Format batch
        T, _ = format_batch(target_batch, pretext_task="rotation")

        # prepare target data
        target_rgb_images = T["rgb"].to(DEVICE)
        target_depth_images = T["depth"].to(DEVICE)
        target_main_labels = T["label"].to(DEVICE)

        outputs = net.forward(target_rgb_images, target_depth_images, mode="main")
        loss_T = criterion(outputs, target_main_labels)

        # compute test stats
        _, preds = torch.max(outputs.data, 1)
        running_corrects = torch.sum(preds == target_main_labels.data).data.item()
        test_main_acc_val += running_corrects/len(target_main_labels)
        test_main_loss_val += loss_T.item()
        del target_depth_images, target_rgb_images, target_main_labels
        n_iters += 1
      
      test_loss.append(test_main_loss_val/n_iters)
      test_acc.append(test_main_acc_val/n_iters)

      print("test main target accuracy: ", test_main_acc_val/n_iters)
      print("test main target loss: ", test_main_loss_val/n_iters)

    print()

    #apply decay policy
    if use_fixed_stepsize == False and iter_count % (next_update_epoch) == 0:
      for g in optimizer.param_groups:
        g['lr'] = lr*curr_gamma
      lr = lr*curr_gamma
      slow_schedule_count += 1
      next_update_epoch = slow_schedule_count**decay_policy
    else: # else use fixed step size
      scheduler.step()
    iter_count += 1

  return net, train_loss, train_acc, validation_loss, validation_acc, test_loss, test_acc

## ROD and synROD - rotation

In [0]:
tfConfig = TransformConfig(resize_shape=299, centercrop_shape=299)   
# config types are imagenet, rgb_mod, depth_mod, rgb_depth_mod                              
synrod_param_values, rod_param_values = tfConfig.get_rotation_configuration(config_type="imagenet")    # mod corresponds to the modification of imagenet weights with the computed ones

In [0]:
synrod_train = SynRODMOD(synrod_path,
                split_path="/content/synARID_50k-split_sync_train1.txt",
                item_extractor_fn="rotation",
                item_extractor_param_values= synrod_param_values,
                )

synrod_test = SynRODMOD(synrod_path,
                split_path="/content/synARID_50k-split_sync_test1.txt",
                item_extractor_fn="rotation",
                item_extractor_param_values= synrod_param_values,
                )

rod = RODMOD(rod_path,
              split_path="/content/rod-split_sync.txt",
              item_extractor_fn="rotation",
              item_extractor_param_values=rod_param_values,
              )

## tuning

In [0]:
parameters_dict = { "lr" : [0.0003, 0.0001],
                    "batch_size":[8],
                   "gamma":[0.10, 0.20],
                   "epochs":[10],
                   "weight_decay":[5e-2],
                    "step_size": [1, 3],
                   "lambda": [1.0],
                   "lr_epoch_num_decay_policy" : [1, 2, 3, None] # 1 == lr reduction after every epoch   used for fast decay
                                                          #  2 == lr reduction at "positive natural number squared"  epochs[ 1-4-9-16-...] slower decay
                                                          #  3 == lr reduction at "positive natural number cubed"  epochs[ 1-8-27-64-...]  slowest decay
                                                          #  None == lr reduction every "stepsize number" of  epochs (step size 3 -> update at epoch [3-6-9-12-...])
                   }
paramGrid = ParameterGrid(parameters_dict)
paramgrid_list = [grid_val for grid_val in paramGrid]


log_file = "variation_tuning.csv"
SAVE_FOLDER = "/content/drive/My Drive/DL_project/logs/"
if not os.path.exists(SAVE_FOLDER):
  os.makedirs(SAVE_FOLDER)
SAVE_PATH = SAVE_FOLDER + log_file
# Create or overwrite file with heading
with open(SAVE_PATH, "w") as f:
  header = ['lr', 'batch_size', 'epochs', 'weight_decay', 'step_size', 'lambda', "train_accuracy", "train_loss", "rod_accuracy", "rod_loss"]
  f.write(", ".join(header) + '\n')
  f.close()

# Now open in append mode
f = open(SAVE_PATH, "a")

for grid in paramgrid_list:
    print("CONFIG:")
    print(grid)

  # Train net
    net, train_loss, train_acc, validation_loss, validation_acc, test_loss, test_acc = train_test_ours(synrod_train, synrod_test, rod, grid, light_validation=False)
  
  # Append on file
    config_data = [grid['lr'], grid['batch_size'], grid['epochs'], grid['weight_decay'], grid['step_size'], grid['lambda'],
                  train_acc[-1], train_loss[-1], test_acc[-1], test_loss[-1]]
    f.write(", ".join([str(el) for el in config_data]) + '\n')
    f.flush()
    model_save_path = os.path.join(SAVE_FOLDER,  "_".join([str(el) for el in config_data]) + ".pth"  )
    torch.save(net.state_dict(), model_save_path)
    plot_title = "train and source validation"
    learning_curves(train_acc, train_loss, validation_acc, validation_loss,plot_title , plot_size=(16,6))
    plot_title = "train and target validation"
    learning_curves(train_acc, train_loss, test_acc, test_loss,plot_title , plot_size=(16,6))
f.close()